# Preparing data for the coherent pipeline
A few steps need to be done on the data before it can be used, namely:
- Take the average over the block - used for later analysis on the CPU
- Multiply the input data the calibration coefficients
- Subtract the continuum
- Sum the 2 polarisations

These are pretty simple. We write the code down here but also do it coherentprep.py

Note: The ordering in memory is not yet decided - there are all sorts of possibilities that we haven't explored yet. it's quite possible hardware gives us ordering that's really weird.

For the sake of this notebook, we put things in (baseline, channel, polarisation, time) order (time going fastest). But as I said, it might well change. The prepare step may well need to do a tranpose or re-ordering of the data on the way through.

In [2]:
%matplotlib inline
import numpy as np
from pylab import *


In [16]:
# Telescope information - more or less fixed
nchan = 288 # number of channels 
nant = 30# number of antennas
nbl = nant*(nant-1)/2 # number of baselines
npol = 2 # number of polariosations
nt = 256 # Number of samples per block
inshape = (nbl, nchan, npol, nt)
dtype = np.dtype('complex64')

print('input shape={} size={} Mbytes'.format(inshape, np.prod(inshape)*dtype.itemsize/1e6))


input shape=(435, 288, 2, 256) size=513.14688 Mbytes


In [8]:
def complex_randn(shape, dtype):
    dout = (np.random.randn(*shape) + 1j*np.random.randn(*shape)).astype(dtype)
    return dout

In [33]:
# These data come in from the CPU - note the shapes and type is complex
calibration_data = np.ones((nbl, nchan, npol), dtype=dtype) # all 1 by defult
sky_model = np.zeros((nbl, nchan, npol), dtype=dtype) # 0 by default - polarised sky model
input_data = complex_randn((nbl, nchan, npol, nt), dtype) # make it random for now.


In [47]:
def do_prepare(input_data, calibration_data, sky_model):
    # make output data using numpy broadcasting rules - it's easy to read by doesn't really explain 
    # wht's going on under the hood, but you get the idea
    output_data = input_data * calibration_data[:,:,:,None] - sky_model[:,:,:,None]
    
    # now do the polarisation sum
    output_data = output_data.sum(axis=2)

    # average for cpu - just keep this. Something will copy it back to the CPU when the kernel is finished
    block_average = input_data.mean(axis=3)
    
    return (output_data, block_average)

In [35]:
dout, davg = do_prepare(input_data, calibration_data, sky_model)
assert dout.shape == (nbl, nchan, nt)
assert davg.shape == (nbl, nchan, npol)

In [37]:
%timeit  do_prepare(input_data, calibration_data, sky_model)

1 loop, best of 3: 984 ms per loop


In [48]:
# This spelled-out look explains the guts of what's going on in more detail.
# It might even go faster too with numba.jit

from numba import jit
@jit(nopython=True)
def do_prepare_numba(input_data, calibration_data, sky_model, output_data, block_average):
    # make output data using numpy broadcasting rules - cheating slightly, but you get the idea
    nbl, nchan, npol, nt = input_data.shape
    for ibl in xrange(nbl):
        for ichan in xrange(nchan):
            for ipol in xrange(npol):
                for it in xrange(nt):
                    d = input_data[ibl, ichan, ipol, it] # read input sample
                    
                    # Output is the sum over polarisations after applying the calibration and subtracting the sky model
                    # Note these are complex multiplications and sums
                    output_data[ibl, ichan, it] += d * calibration_data[ibl, ichan, ipol] - sky_model[ibl, ichan, ipol ]
                    
                    # block average is the average over all times
                    block_average[ibl, ichan, ipol] += d
    
    return (output_data, block_average)

In [46]:
dout *= 0
davg *= 0
%timeit do_prepare_numba(input_data, calibration_data, sky_model, dout, davg)

1 loop, best of 3: 235 ms per loop


In [44]:
# Wow - numba is 5x faster than naked numpy

In [52]:
# here it is with the code 
import coherentprep
reload(coherentprep)
prep = coherentprep.Prepare(nt, calibration_data, sky_model)
dout = prep(input_data)